## Imports

In [1]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob

## Read-in

In [2]:
# Get today's date
today = datetime.now()

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Mar. 21st Mar. 28th


In [3]:
# filename = glob.glob('redfin*.csv')
# filename = filename[0]

In [4]:
df = pd.read_csv('weekly_top_ten.csv')
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]
df = df.sort_values(by='PRICE',ascending=False)
df_top_ten = df.head(10)

In [6]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
107,PAST SALE,March-18-2024,Condo/Co-op,50 S Pointe Dr #1805,Miami Beach,FL,33139.0,6250000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/50-S-Poi...,MARMLS,A11531852,N,Y,25.767875,-80.132756
106,PAST SALE,March-18-2024,Condo/Co-op,50 S Pointe Dr #2305,Miami Beach,FL,33139.0,5750000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/50-S-Poi...,MARMLS,A11515879,N,Y,25.767875,-80.132756
117,PAST SALE,March-18-2024,Condo/Co-op,15701 Collins Ave #1401,Sunny Isles Beach,FL,33160.0,4400000.0,3.0,4.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami-Beach/15...,MARMLS,A11268836,N,Y,25.921979,-80.121703
89,PAST SALE,March-22-2024,Condo/Co-op,10 Venetian Way #2201,Miami Beach,FL,33139.0,3275000.0,3.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/10-Venet...,MARMLS,A11496960,N,Y,25.791132,-80.146640
43,PAST SALE,March-25-2024,Condo/Co-op,6000 Island Blvd #2806,Aventura,FL,33160.0,2830000.0,4.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/6000-Island...,Beaches MLS,RX-10933840,N,Y,25.942792,-80.135400
18,PAST SALE,March-22-2024,Condo/Co-op,5775 Collins Ave #1205,Miami Beach,FL,33140.0,2750000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/5775-Col...,MARMLS,A11482601,N,Y,25.839257,-80.120395
16,PAST SALE,March-20-2024,Condo/Co-op,789 Crandon Blvd #902,Key Biscayne,FL,33149.0,2650000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/789-Cra...,MARMLS,A11410913,N,Y,25.686725,-80.159125
112,PAST SALE,March-20-2024,Condo/Co-op,18975 Collins Ave #1904,Sunny Isles Beach,FL,33160.0,2625000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11469312,N,Y,25.951740,-80.119755
74,PAST SALE,March-20-2024,Condo/Co-op,1000 Brickell Plz Ph 5707,Miami,FL,33131.0,2500000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1000-Brickell-...,MARMLS,A11410955,N,Y,25.764264,-80.192725
7,PAST SALE,March-21-2024,Condo/Co-op,3300 NE 188th St #616,Aventura,FL,33180.0,2335000.0,3.0,4.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/3300-NE-188th-...,MARMLS,A11461277,N,Y,25.948955,-80.135269


## Current Week's Values

In [7]:
# Number of condos sold
current_week_condo_sales = len(df)
print(len(df))

137


In [8]:
df

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
107,PAST SALE,March-18-2024,Condo/Co-op,50 S Pointe Dr #1805,Miami Beach,FL,33139.0,6250000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/50-S-Poi...,MARMLS,A11531852,N,Y,25.767875,-80.132756
106,PAST SALE,March-18-2024,Condo/Co-op,50 S Pointe Dr #2305,Miami Beach,FL,33139.0,5750000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/50-S-Poi...,MARMLS,A11515879,N,Y,25.767875,-80.132756
117,PAST SALE,March-18-2024,Condo/Co-op,15701 Collins Ave #1401,Sunny Isles Beach,FL,33160.0,4400000.0,3.0,4.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami-Beach/15...,MARMLS,A11268836,N,Y,25.921979,-80.121703
89,PAST SALE,March-22-2024,Condo/Co-op,10 Venetian Way #2201,Miami Beach,FL,33139.0,3275000.0,3.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/10-Venet...,MARMLS,A11496960,N,Y,25.791132,-80.146640
43,PAST SALE,March-25-2024,Condo/Co-op,6000 Island Blvd #2806,Aventura,FL,33160.0,2830000.0,4.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/6000-Island...,Beaches MLS,RX-10933840,N,Y,25.942792,-80.135400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,PAST SALE,March-19-2024,Condo/Co-op,11925 NE 2nd Ave Unit B101,North Miami,FL,33161.0,175000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/11925-NE-2nd-A...,MARMLS,A11517565,N,Y,25.885395,-80.193407
62,PAST SALE,March-22-2024,Condo/Co-op,7360 SW 82 St Unit E202,Miami,FL,33143.0,175000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/7360-SW-82nd-S...,MARMLS,A11520612,N,Y,25.693450,-80.313140
128,PAST SALE,March-20-2024,Condo/Co-op,2903 Point East Dr Unit K610,Aventura,FL,33160.0,170000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/2903-Point-...,MARMLS,A11521482,N,Y,25.940289,-80.146831
127,PAST SALE,March-22-2024,Condo/Co-op,1680 NE 191st St Unit 218-2,Miami,FL,33179.0,135000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1680-NE-191st-...,MARMLS,A11502959,N,Y,25.949477,-80.168976


In [9]:
# Average sale price
average_sale_price = round(df['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$793,404


In [10]:
# Average PSF
average_price_psf = round(df['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$599


In [11]:
# Total sales dollar volume
current_week_condo_sales_total = df['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$108,696,400


In [12]:
filename_previous_week = glob.glob('redfin*.csv')

In [13]:
df_list = []
for file in filename_previous_week:
    df_x = pd.read_csv(file)
    df_list.append(df_x)
    
all_data = pd.concat(df_list)

In [14]:
all_data = all_data.dropna(subset='SOLD DATE')

In [15]:
# Convert 'SOLD DATE' to datetime format
all_data['SOLD DATE'] = pd.to_datetime(all_data['SOLD DATE'], format='%B-%d-%Y')

# To filter between specific dates, ensure these are also in a compatible format or converted to datetime objects
start_date = pd.to_datetime('2024-03-10')
end_date = pd.to_datetime('2024-03-18')

# Filter DataFrame for dates between Mar. 10th, 2024, and Mar. 18th, 2024
second_week_data = all_data[(all_data['SOLD DATE'] > start_date) & (all_data['SOLD DATE'] < end_date)]

In [16]:
second_week_data['$/SQUARE FEET']

2       538.0
7       257.0
14      241.0
24      223.0
25      280.0
        ...  
264    1003.0
267     667.0
273     364.0
274    3024.0
276     681.0
Name: $/SQUARE FEET, Length: 183, dtype: float64

In [17]:
print(len(second_week_data))
print('-------------------')
print(second_week_data['PRICE'].mean())
print('-------------------')
print(second_week_data['$/SQUARE FEET'].mean())
print('-------------------')
print(second_week_data['PRICE'].sum())

183
-------------------
1015318.868852459
-------------------
601.9234972677596
-------------------
185803353.0


## Previous Week's Values

In [18]:
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: $XX million -- AS A FLOAT VALUE)')
previous_week_condo_sales_total = input()

Input Previous Week Condo Sales Total (number units sold):

Input Previous Week Condo Average Sales Price:

Input Previous Week Condo Average PSF:

Input Previous Week Condo Sales total (ex: $XX million -- AS A FLOAT VALUE)



## Clean Data

In [19]:
# df_top_ten

## Correction section

In [20]:
# df.at[index#,'col_name']

## Format Data

In [21]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [22]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [23]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'RANK', 'COLOR'],
      dtype='object')

In [24]:
pd.set_option('display.max_columns',None)

In [25]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,RANK,COLOR
107,PAST SALE,March-18-2024,Condo/Co-op,50 S Pointe Dr #1805,Miami Beach,FL,33139.0,6250000.0,2.0,2.5,CONTINUUM ON SOUTH BEACH,1606.0,NaN,2008.0,NaN,3892.0,4165.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/50-S-Poi...,MARMLS,A11531852,N,Y,25.767875,-80.132756,1,orange


In [26]:
def popup_html(row):
    building_name = row['LOCATION']
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(building_name) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [27]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} - {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [28]:
m.save('index.html')

## Data snagger

In [29]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [30]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,RANK,COLOR
107,PAST SALE,March-18-2024,Condo/Co-op,50 S Pointe Dr #1805,Miami Beach,FL,33139.0,6250000.0,2.0,2.5,CONTINUUM ON SOUTH BEACH,1606.0,NaN,2008.0,NaN,3892.0,4165.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/50-S-Poi...,MARMLS,A11531852,N,Y,25.767875,-80.132756,1,orange


In [31]:
# , {df_top_ten[df_top_ten['PRICE'].idxmax()]['ADDRESS']} | Price ${df_top_ten[df_top_ten['PRICE'].idxmax()]['PRICE']:,.0f}

In [32]:
df_top_ten['PRICE'].iloc[0]

6250000.0

In [33]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [34]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/condo_sales_week_ending_032524


# CREATE TEMPLATE 

In [35]:
muni_set = set(df_top_ten['CITY'])

In [36]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [37]:
df_top_ten.reset_index(inplace=True,drop=True)

In [38]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [39]:
top_sale

'CONTINUUM ON SOUTH BEACH at 50 S Pointe Dr #1805 in Miami Beach'

In [40]:
previous_week_condo_sales_total = 185_000_000

In [41]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

ValueError: could not convert string to float: ''

In [ ]:
# df_top_ten

## RESUME HERE

In [ ]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} tops Miami-Dade weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f}M to ${df_top_ten.at[0,'PRICE']:,.0f}M
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m Miami-Dade County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m Miami-Dade County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Adam Farence
\033[1mRESEARCH:\033[0m 
\033[1mSocial:\033[0m #MiamiDade condo sales XXXXX
\033[1mART:\033[0m

Interactive map

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, Miami-Dade County, weekly condo sales, {', '.join(str(x) for x in muni_set)}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

Miami-Dade County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} sales average from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)

In [ ]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [ ]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [ ]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf")

print(story_checklist)

In [ ]:
print(df_top_ten['URL'].iloc[0])

In [ ]:
print(df_top_ten['URL'].iloc[-1])

In [ ]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)


In [ ]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)


In [ ]:
df_top_ten

## Time on Market Calculator

In [ ]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2023, 11, 29) ## List (Earlier) date
date2 = datetime(2024, 3, 21) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)